In [3]:
from langchain_groq import ChatGroq
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
llm = ChatGroq(temperature=0, model="llama-3.3-70b-versatile", max_tokens=4000,api_key='')

In [5]:
chat_store = {}
long_term_memory = {}


In [6]:
def get_chat_history(session_id: str):
    if session_id not in chat_store:
        chat_store[session_id] = ChatMessageHistory()
    return chat_store[session_id]

def update_long_term_memory(session_id: str, input: str, output: str):
    if session_id not in long_term_memory:
        long_term_memory[session_id] = []
    if len(input) > 20:  # Simple logic: store inputs longer than 20 characters
        long_term_memory[session_id].append(f"User said: {input}")
    if len(long_term_memory[session_id]) > 5:  # Keep only last 5 memories
        long_term_memory[session_id] = long_term_memory[session_id][-5:]

def get_long_term_memory(session_id: str):
    return ". ".join(long_term_memory.get(session_id, []))

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the information from long-term memory if relevant."),
    ("system", "Long-term memory: {long_term_memory}"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [8]:
chain = prompt | llm
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [9]:
def chat(input_text: str, session_id: str):
    long_term_mem = get_long_term_memory(session_id)
    response = chain_with_history.invoke(
        {"input": input_text, "long_term_memory": long_term_mem},
        config={"configurable": {"session_id": session_id}}
    )
    update_long_term_memory(session_id, input_text, response.content)
    return response.content

In [10]:
session_id = "user_123"

print("AI:", chat("Hello! My name is Alice.", session_id))
print("AI:", chat("What's the weather like today?", session_id))
print("AI:", chat("I love sunny days.", session_id))
print("AI:", chat("Do you remember my name?", session_id))

AI: Hello Alice, it's nice to meet you. Is there something I can help you with or would you like to chat?
AI: I'm not currently able to access real-time information or your location, Alice. However, I can suggest some ways for you to find out the weather. You can check a weather website or app on your device, or tune into a local news station for the latest forecast. Would you like some recommendations for weather websites or apps?
AI: Sunny days can be so uplifting and energizing, can't they, Alice? There's something about the warmth and brightness of the sun that just makes everything feel more vibrant and alive. Do you have any fun plans or activities that you like to do on sunny days?
AI: Your name is Alice. I remember you introduced yourself to me at the beginning of our conversation.


In [11]:
print("Conversation History:")
for message in chat_store[session_id].messages:
    print(f"{message.type}: {message.content}")

print("\nLong-term Memory:")
print(get_long_term_memory(session_id))

Conversation History:
human: Hello! My name is Alice.
ai: Hello Alice, it's nice to meet you. Is there something I can help you with or would you like to chat?
human: What's the weather like today?
ai: I'm not currently able to access real-time information or your location, Alice. However, I can suggest some ways for you to find out the weather. You can check a weather website or app on your device, or tune into a local news station for the latest forecast. Would you like some recommendations for weather websites or apps?
human: I love sunny days.
ai: Sunny days can be so uplifting and energizing, can't they, Alice? There's something about the warmth and brightness of the sun that just makes everything feel more vibrant and alive. Do you have any fun plans or activities that you like to do on sunny days?
human: Do you remember my name?
ai: Your name is Alice. I remember you introduced yourself to me at the beginning of our conversation.

Long-term Memory:
User said: Hello! My name is A